In [1]:
!pip install fuzzywuzzy distance

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
import numpy as np

# Tools for preprocessing input data
from bs4 import BeautifulSoup
from nltk import word_tokenize
#from nltk.corpus import stopword
from nltk.stem import WordNetLemmatizer
import re


import seaborn as sns
# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
#from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import distance

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
import tensorflow as tf
from keras.engine.topology import Layer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Embedding,Bidirectional,LSTM,Attention,Concatenate,GlobalAveragePooling1D,Dropout, Lambda
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

train_df = pd.read_csv( "/content/drive/MyDrive/Kaggle/train_preprocessed.csv") 
embeddings_index = pickle.load( open( "/content/drive/MyDrive/Kaggle/embeddings_index.p", "rb" ) )

In [5]:
max_num_words = 200000
sequence_length = 60
embedding_dim = 300
validation_split_ratio = 0.1

In [6]:
train_df['question1_clean']=train_df['question1_clean'].astype(str)
train_df['question2_clean']=train_df['question2_clean'].astype(str)

In [7]:
train_df = train_df.drop(train_df.loc[(train_df.q1_n_words>20 )| (train_df.q2_n_words>20)].index)

In [8]:
train_df.describe()

,id,qid1,qid2,is_duplicate,q1len,q2len,q1_n_words,q2_n_words,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,word_share
count,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000,357497.000000
mean,202145.121061,214322.597345,216853.860172,0.398854,53.344657,52.560542,9.779293,9.740722,65.897767,64.118672,66.507305,0.391376,0.250178
std,116687.525368,157392.705709,159314.466917,0.489663,21.081838,20.956088,3.620838,3.647376,16.211599,17.616344,16.476694,0.219632,0.126585
min,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,101091.000000,72189.000000,71569.000000,0.000000,38.000000,38.000000,7.000000,7.000000,54.000000,50.000000,53.000000,0.222222,0.157895
50%,202176.000000,187485.000000,190624.000000,0.000000,49.000000,48.000000,9.000000,9.000000,66.000000,64.000000,67.000000,0.350000,0.250000
75%,303229.000000,342389.000000,348676.000000,1.000000,65.000000,64.000000,12.000000,12.000000,78.000000,78.000000,79.000000,0.526316,0.352941
max,404289.000000,537932.000000,537933.000000,1.000000,224.000000,235.000000,20.000000,20.000000,100.000000,100.000000,100.000000,0.992481,0.500000


In [9]:
q1 = train_df['question1_clean'].values
q2 = train_df['question2_clean'].values
tokenizer = Tokenizer(num_words = max_num_words, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(q1+q2)

# this takes our sentences and replaces each word with an integer
X_1 = tokenizer.texts_to_sequences(q1)
X_2 = tokenizer.texts_to_sequences(q2)

# we then pad the sequences so they're all the same length (sequence_length)
X_1 = pad_sequences(X_1, int(sequence_length))
X_2 = pad_sequences(X_2, int(sequence_length))

y = train_df['is_duplicate']

In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 75120 unique tokens.


In [11]:
embedding_matrix = np.zeros((max_num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))


Null word embeddings: 


In [12]:
q1_n_words=train_df['q1_n_words']
q2_n_words=train_df['q2_n_words']
have_same_q_w=train_df['have_same_q_w']

In [13]:
q1_n_words=q1_n_words.values
q2_n_words=q2_n_words.values
have_same_q_w=have_same_q_w.values

In [14]:
scaler = MinMaxScaler()
scaler.fit(q1_n_words.reshape(-1,1))
q1_n_words=scaler.transform(q1_n_words.reshape(-1,1))

scaler.fit(q2_n_words.reshape(-1,1))
q2_n_words=scaler.transform(q2_n_words.reshape(-1,1))

scaler.fit(have_same_q_w.reshape(-1,1))
have_same_q_w=scaler.transform(have_same_q_w.reshape(-1,1))


In [15]:
def exponent_neg_manhattan_distance(left, right):
    ''' 
    Purpose : Helper function for the similarity estimate of the LSTMs outputs
    Inputs : Two n-dimensional vectors
    Output : Manhattan distance between the input vectors
    
    '''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [19]:
# The visible layer
left_input = Input(shape=(sequence_length,))
right_input = Input(shape=(sequence_length,))

embedding_layer = Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], 
                             weights=[embedding_matrix],input_length = sequence_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(50,dropout=0.2, recurrent_dropout=0.2)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
output = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])


#output = Dense(1, activation="sigmoid")(malstm_distance)



In [20]:
# Combine all of the above in a Model
model = Model([left_input, right_input], [output])

In [21]:
model.compile(loss='mse', optimizer= 'nadam', metrics=['accuracy'])

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 60, 300)      60000000    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 50)           70200       embedding_2[0][0]            

In [ ]:
history = model.fit([X_1, X_2], y, epochs = 30, batch_size = 2048, verbose = 1, validation_split = 0.1, shuffle = True)

Epoch 1/30
 39/158 [======>.......................] - ETA: 12:09 - loss: 0.3681 - accuracy: 0.5992

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()